In [1]:
DATA_PATH =  "..\data\WA_Fn-UseC_-Telco-Customer.pkl"

In [2]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import os
import yaml

import plotly 
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.manifold import TSNE

from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import silhouette_score

pd.options.display.max_rows = 1000

In [3]:
df = pd.read_pickle(DATA_PATH)

## Splitting the data into train and test sets

In [4]:
def object_to_int(dataframe_series):
    
    if dataframe_series.dtype=='object':
        dataframe_series = LabelEncoder().fit_transform(dataframe_series)
        
    return dataframe_series

In [5]:
df = df.apply(lambda x: object_to_int(x))
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,5375,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,2,29.85,29.85,0
1,3962,1,0,0,0,34,1,0,0,2,...,2,0,0,0,1,0,3,56.95,1889.50,0
2,2564,1,0,0,0,2,1,0,0,2,...,0,0,0,0,0,1,3,53.85,108.15,1
3,5535,1,0,0,0,45,0,1,0,2,...,2,2,0,0,1,0,0,42.30,1840.75,0
4,6511,0,0,0,0,2,1,0,1,0,...,0,0,0,0,0,1,2,70.70,151.65,1


In [6]:
df.drop(columns = ['customerID'],inplace=True)

In [7]:
X = df.drop(columns = ['Churn'])
y = df['Churn'].values
X = StandardScaler().fit_transform(X)


In [8]:
num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges','gender']
dff = X[['tenure', 'MonthlyCharges', 'TotalCharges', 'gender']].copy()
std_skills = X.drop(num_cols, axis=1).copy()
#std_skills = StandardScaler().fit_transform(skills_ohe)

C:\Users\AL-FAJR\AppData\Local\Temp\ipykernel_21488\2738910832.py:2: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dff = X[['tenure', 'MonthlyCharges', 'TotalCharges', 'gender']].copy()


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
tsne_projection = TSNE(n_components=2, 
                       perplexity=4,
                       learning_rate=0.01,
                       init='pca', 
                       method='barnes_hut', 
                       n_jobs=2, 
                       n_iter=10**10,
                       random_state=0).fit_transform(std_skills.T)

tsne_projection = pd.DataFrame(tsne_projection, index=skills_ohe.columns)

In [ ]:
fig = px.scatter(x=tsne_projection[0], y=tsne_projection[1], text=tsne_projection.index)
fig.update_traces(textposition='top center')
fig.update_layout(height=1000, width=1000, title_text='TSNE')
fig.show()

In [ ]:
range_n_cluster = list(range(3,7))
silhoutte_score = []
best_cluster_model = None 

for n_clusters in range_n_cluster:
    cluster_model  = AgglomerativeClustering(n_clusters=n_clusters, linkage='ward')
    cluster_labels = cluster_model.fit_predict(tsne_projection)
    
    silhouette_avg = silhouette_score(tsne_projection, cluster_labels)
    silhoutte_score += [silhouette_avg]
    
    if silhouette_avg >= np.max(silhoutte_score):
        best_cluster_model = cluster_model

In [ ]:
plt.plot(range_n_cluster, silhoutte_score)
plt.axvline(best_cluster_model.n_clusters, color='black')

In [ ]:
cluster_labels = ["skills_group_" + str(label) 
                 for label in best_cluster_model.labels_]

In [ ]:
fig = px.scatter(x=tsne_projection[0], 
                 y=tsne_projection[1], 
                 text=tsne_projection.index, 
                 color=cluster_labels)
fig.update_traces(textposition='top center')
fig.update_layout(height=800, width=800, title_text='Cluster')
fig.show()

In [ ]:
# How to better write this
skills_clusters = tsne_projection.index.to_series().groupby(cluster_labels).apply(list)

In [ ]:
for cluster, skills in skills_clusters.items():
    print(cluster)
    print(skills)

In [ ]:
new_features = []

for cluster, skills in skills_clusters.items():
    cluster_sum = skills_ohe[skills].sum(axis=1)
    cluster_sum.name = cluster
    new_features.append(cluster_sum)

fe_clustered_skills = pd.concat(new_features, axis=1)

In [ ]:
fe_clustered_skills

In [ ]:
combined_features_df = pd.concat([fe_clustered_skills, 
                                 dff], 
                                 axis=1)

In [ ]:
combined_features_df

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(combined_features_df, y, test_size=0.3, random_state=0)